<a href="https://colab.research.google.com/github/rakaplay/cnn_recogn/blob/main/ModelNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf

In [ ]:
def prepare_upscaling_data(image_paths, downscale_size=(8, 8), original_tile_size=(16, 16)):
    downscale_images = []
    original_tiles = []

    for image_path in image_paths:
        image = Image.open(image_path)
        image = image.convert('RGB')

        # Вычисляем паддинг
        width, height = image.size
        padding_width = (original_tile_size[0] - width % original_tile_size[0]) % original_tile_size[0]
        padding_height = (original_tile_size[1] - height % original_tile_size[1]) % original_tile_size[1]

        # Добавляем паддинг
        image = image.crop((-padding_width // 2, -padding_height // 2,
                           width + padding_width - padding_width // 2,
                           height + padding_height - padding_height // 2))

        np_image = np.array(image)

        for y in range(0, np_image.shape[0], original_tile_size[1]):
            for x in range(0, np_image.shape[1], original_tile_size[0]):
                original_tile = np_image[y:y+original_tile_size[1], x:x+original_tile_size[0]]
                original_tiles.append(original_tile)
                downscale_image = Image.fromarray(original_tile).resize(downscale_size)
                downscale_images.append(np.array(downscale_image))

    return np.array(downscale_images), np.array(original_tiles)

image_paths = [f"photos/({i}).jpg" for i in range(12)]
images, labels = prepare_upscaling_data(image_paths)

print(f"Количество кусочков 8x8: {len(images)}")
print(f"Количество кусочков 16x16: {len(labels)}")

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
config = tf.compat.v1.ConfigProto(gpu_options=gpu_options)
session = tf.compat.v1.Session(config=config)

img_size = (8, 8)
output_size = (16, 16)

def process_images_from_list(image_paths, input_size=(8, 8), label_size=(16, 16)):
    input_chunks_all = []
    label_chunks_all = []

    input_chunks, label_chunks = prepare_upscaling_data(image_paths, input_size, label_size)

    input_chunks_all.extend(input_chunks)
    label_chunks_all.extend(label_chunks)

    return input_chunks_all, label_chunks_all

input_chunks_all, label_chunks_all = process_images_from_list(image_paths, img_size, output_size)

print("Всего входных кусочков:", len(input_chunks_all))
print("Всего лейблов:", len(label_chunks_all))

input_chunks_all = np.array(input_chunks_all).astype('float32') / 255.0
label_chunks_all = np.array(label_chunks_all).astype('float32') / 255.0

input_chunks_all = input_chunks_all.reshape(-1, 8, 8, 3)
label_chunks_all = label_chunks_all.reshape(-1, 16, 16, 3)

def augment(chunk):
    return chunk * np.random.uniform(0.5, 1, size=chunk.shape)

input_chunks_all = np.array([augment(i) for i in input_chunks_all])
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((8, 8, 3)),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2DTranspose(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2DTranspose(768, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')
])

model.compile(optimizer='adam', loss='mae', metrics=['accuracy'])
model.summary()

model.fit(input_chunks_all, label_chunks_all, epochs=10, batch_size=256)

Количество кусочков 8x8: 190280
Количество кусочков 16x16: 190280
Всего входных кусочков: 190280
Всего лейблов: 190280
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 up_sampling2d_3 (UpSamplin  (None, 16, 16, 3)         0         
 g2D)                                                            
                                                                 
 conv2d_transpose_8 (Conv2D  (None, 16, 16, 256)       7168      
 Transpose)                                                      
                                                                 
 conv2d_transpose_9 (Conv2D  (None, 16, 16, 768)       1770240   
 Transpose)                                                      
                                                                 
 conv2d_transpose_10 (Conv2  (None, 16, 16, 3)         20739     
 DTranspose)                                                     
 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
!cp drive/MyDrive/saves.tar model.tar
!tar -xf model.tar

In [ ]:
model.save('upscale3.keras')

In [ ]:
import requests
a = requests.get("https://nekofile.eu.org/5711d1b5863602eb4b1e4vifu")
file = open("img.jpg", "wb")
file.write(a.content)
file.close()

In [ ]:
import cv2
import tensorflow as tf
import numpy as np
from PIL import Image




gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.2)
config = tf.compat.v1.ConfigProto(gpu_options=gpu_options)
session = tf.compat.v1.Session(config=config)




# Загрузка изображения из файла
input_image_path = 'imgg.png'  # Замени на путь к твоему входному файлу
output_image_path = 'imggg.png'  # Замени на путь для сохранения результата






# Загрузка модели
model = tf.keras.models.load_model('upscale3.keras')



def split_image_to_tiles(image_data, tile_size=(8, 8)):
    """
    Разделяет изображение на кусочки заданного размера.
    """
    tiles = []
    height, width = image_data.shape[:2]

    # Вычисляем новую ширину и высоту, кратные размеру тайла
    new_width = ((width + tile_size[1] - 1) // tile_size[1]) * tile_size[1]
    new_height = ((height + tile_size[0] - 1) // tile_size[0]) * tile_size[0]

    # Создаем новое изображение с черным фоном
    new_image = np.zeros((new_height, new_width, 3), dtype=np.uint8)

    # Вычисляем координаты для вставки исходного изображения
    x_offset = (new_width - width) // 2
    y_offset = (new_height - height) // 2

    # Вставляем исходное изображение в новое
    new_image[y_offset:y_offset+height, x_offset:x_offset+width] = image_data

    # Разделяем на тайлы
    tiles = []
    for y in range(0, new_height, tile_size[0]):
        for x in range(0, new_width, tile_size[1]):
            tile = new_image[y:y+tile_size[0], x:x+tile_size[1]]
            tiles.append(tile)

    return tiles, (new_height, new_width) # Возвращаем новый размер

#def split_image_to_tiles(image_data, tile_size=(8, 8)):
#    """
#    Разделяет изображение на кусочки заданного размера.
#    """
#
#    tiles = []
#    height, width = image_data.shape[:2]
#    print(height, width)
#    height, width = (height // 8) * 8, (width // 8) * 8
#    image_data = cv2.resize(image_data, (height, width), cv2.INTER_LINEAR)
#    Image.fromarray(image_data).save('watafa.png', 'PNG')
#    print(image_data.shape[:2])
#    for y in range(0, height, tile_size[0]):
#        for x in range(0, width, tile_size[1]):
#            tile = image_data[y:min(y+tile_size[0], height), x:min(x+tile_size[1], width)]
#            if tile.size > 0:
#                tiles.append(tile)
#    return tiles, (height, width)




import cv2
import numpy as np

def upscale_tiles(tiles, model, batch_size=16384):
    """
    Применяет модель апскейлинга к каждому кусочку, обрабатывая по batch_size тайлов за раз.
    """
    upscaled_tiles = []
    total_tiles = len(tiles)
    processed_tiles = 0

    for i in range(0, total_tiles, batch_size):
        batch_tiles = tiles[i: min(i + batch_size, total_tiles)]

        # Преобразуем список тайлов в массив NumPy
        batch_tiles = np.array(batch_tiles)

        # Нормализация тайлов
        batch_tiles = batch_tiles.astype('float32') / 255.0

        # Предсказание модели
        batch_upscaled_tiles = model.predict(batch_tiles)

        # Преобразование обратно в формат изображения
        batch_upscaled_tiles = (batch_upscaled_tiles * 255).astype(np.uint8)

        # Добавляем обработанные тайлы в список
        upscaled_tiles.extend(batch_upscaled_tiles)

        processed_tiles += len(batch_tiles)
        print(f"Обработано {processed_tiles} из {total_tiles} тайлов")

    return upscaled_tiles


def merge_tiles(tiles, original_image_size, tile_size=(16, 16)):  # Изменено tile_size
    """
    Собирает апскейленные кусочки обратно в изображение.
    """
    # Увеличиваем размер выходного изображения в два раза
    upscaled_image = np.zeros((original_image_size[0] * 2, original_image_size[1] * 2, 3), dtype=np.uint8)
    i = 0
    for y in range(0, upscaled_image.shape[0], tile_size[0]):
        for x in range(0, upscaled_image.shape[1], tile_size[1]):
            if i < len(tiles):
                # Изменяем размер тайла перед помещением в область изображения
                upscaled_image[y:y+tile_size[0], x:x+tile_size[1]] = tiles[i][:tile_size[0], :tile_size[1]]
                i += 1
    return upscaled_image

def bring_back_aspect_ratio(image, original_size):
    """
    Изменяет размер изображения до исходных пропорций.
    """
    original_height, original_width = original_size
    resized_image = cv2.resize(image, (original_width * 2, original_height * 2), interpolation=cv2.INTER_CUBIC)
    return resized_image


# Загрузка изображения
image = Image.open(input_image_path).convert('RGB') # Убираем альфа-канал
original_image = np.array(image)

# Обработка изображения
tiles, new_size = split_image_to_tiles(original_image) # Получаем новый размер
upscaled_tiles = upscale_tiles(tiles, model)
upscaled_image = merge_tiles(upscaled_tiles, new_size) # Используем новый размер
upscaled_image = bring_back_aspect_ratio(upscaled_image, original_image.shape[:2])

# Сохранение апскейленного изображения
upscaled_image_pil = Image.fromarray(upscaled_image)
upscaled_image_pil.save(output_image_path)

print(f"Апскейленное изображение сохранено в: {output_image_path}")

512/512 [==============================] - 1s 2ms/step
Обработано 16384 из 55620 тайлов
512/512 [==============================] - 1s 2ms/step
Обработано 32768 из 55620 тайлов
512/512 [==============================] - 1s 2ms/step
Обработано 49152 из 55620 тайлов
203/203 [==============================] - 1s 2ms/step
Обработано 55620 из 55620 тайлов
Апскейленное изображение сохранено в: imggg.png


In [ ]:
model.save("md.keras")